dropout

In [6]:
import torch
import torchvision
import random

device = 'cuda' if torch.cuda.is_available() else 'cpu'
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed(777)

In [7]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100
drop_prob = 0.5

In [8]:
mnist_train = torchvision.datasets.MNIST(root = 'MNIST_data/', train=True, transform=torchvision.transforms.ToTensor(), download=True)
mnist_test = torchvision.datasets.MNIST(root = 'MNIST_data/', train=False, transform=torchvision.transforms.ToTensor(), download=True)


In [9]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)

In [10]:
linear1 = torch.nn.Linear(784, 512, bias=True).to(device)
linear2 = torch.nn.Linear(512, 512, bias=True).to(device)
linear3 = torch.nn.Linear(512, 512, bias=True).to(device)
linear4 = torch.nn.Linear(512, 512, bias=True).to(device)
linear5 = torch.nn.Linear(512, 10, bias=True).to(device)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=drop_prob)
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

model = torch.nn.Sequential(linear1, relu, dropout, 
                            linear2, relu, dropout, 
                            linear3, relu, dropout, 
                            linear4, relu, dropout, 
                            linear5).to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


dropout은 학습할 때(train)만 노드들을 무작위로 선택하고, 그렇지 않은 때(eval)는 노드들을 모두 사용하기 때문에
model.train()과 model.eval() 모드를 잘 구분해서 사용해야 함.

model.train() 시에 model 안에서 dropout을 사용해 노드들을 선택해 사용하겠다는 의미

model.eval() 시에 model 안에서 dropout을 사용하지 않고 모든 노드를 사용하겠다는 의미

In [11]:
model.train()
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.view(-1, 28*28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch
    
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
print('Learning finished')

Epoch: 0001 cost = 0.484163672
Epoch: 0002 cost = 0.218449965
Epoch: 0003 cost = 0.177711502
Epoch: 0004 cost = 0.154004127
Epoch: 0005 cost = 0.137513146
Epoch: 0006 cost = 0.133320808
Epoch: 0007 cost = 0.122347280
Epoch: 0008 cost = 0.122127667
Epoch: 0009 cost = 0.108159319
Epoch: 0010 cost = 0.102252863
Epoch: 0011 cost = 0.104724124
Epoch: 0012 cost = 0.101490669
Epoch: 0013 cost = 0.094306141
Epoch: 0014 cost = 0.090632297
Epoch: 0015 cost = 0.088698171
Learning finished


In [12]:
with torch.no_grad():
    model.eval()
    X_test = mnist_test.data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.targets.to(device)
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test

    accuracy = correct_prediction.float().mean()
    print('Accuracy :', accuracy.item())
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.data[r].view(-1, 28*28).float().to(device)
    Y_single_data = mnist_test.targets[r].view(-1, 1).to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())


Accuracy : 0.9786999821662903
Label:  8
Prediction:  3
